# Simulating Images with Unstacked Mirror Segments

For the earliest steps of JWST OTE commissioning (OTE-01 through OTE-05), the mirror segments will be scattered across the JWST FOV. It is not until OTE-06 that the segments are moved into the small array that fits on a single NIRCam detector.

<div class="alert alert-block alert-warning">
    
<b>Note:</b> <br>

This notebook describes how to use MIRaGe to simulate images with *grossly* unstacked mirror states - specifically, states after initial deployment but before OTE-06. For simulations of unstacked mirror states after OTE-06, please refer to the [Nonnominal PSF example notebook](Nonnominal_PSF_simulations_use_examples.ipynb).

</div>

In this notebook, we demonstrate how use MIRaGe to simulate images from those early commissioning steps. The process is as follows:
- Parse APT output files to access the details and structure of a given program
- Perturb the primary and secondary mirrors to reflect expectations for initial deployment (either generate perturbations randomly or load them from a defined mirror state)
- Record the tilt of each PM, and remove the piston and tilt
- Generate PSF libraries from the perturbed mirror state with piston/tip/tilt removed for each mirror segment
- Generate MIRaGe YAML input files that include:
    - A flag, `expand_catalog_for_segments`, that tells MIRaGe to expect 18 separate segment PSF libraries
    - The directory where to find the special segment PSF libraries
- Generate seed images from those YAMLs using a process that:
    - Iterates over all the point sources in the detector FOV
    - Iterates over the 18 segment PSFs and calculates that segment's location by adding its tilt offset to the original point source location
    - If an offset point source falls within the detector FOV, places the corresponding segment PSF on the image
- Follow the nominal procedures for adding dark exposure and detector effects

### Table of Contents:
1. [Export program information from APT](#export_apt)
2. [Generate the perturbed OTE state](#generate_ote)
3. [Create PSF library files](#make_psf_libs)
4. [Create `.yaml` files for each exposure](#make_yamls)
5. [Generate the simulated image](#simulate_images)

Appendix A: [Generating data for an entire observation](#simulate_whole_obs)

Appendix B: [Combining data into a mosaic](#mosaic)

## Import necessary packages and modules

In [ ]:
# Standard Library Imports
from glob import glob
import os

# Third Party Imports
from astropy.io import ascii as asc
from astropy.io import fits
from matplotlib import cm
import matplotlib.pyplot as plt
import numpy as np
import pysiaf
import webbpsf

# Local Imports
from mirage import imaging_simulator
from mirage.catalogs import get_catalog
from mirage.psf import psf_selection, deployments, segment_psfs
from mirage.utils.utils import ensure_dir_exists
from mirage.yaml import yaml_generator

# View matplotlib plots inline
%matplotlib inline

---
<a id='export_apt'></a>
# 1. Export Program Information from APT

MIRaGe requires APT program output files in order to generate data with unstacked PSFs.

### Get needed files from APT program

Open the APT file for the program you want to simulate. If you don't have the file locally, you can load this program in APT by selecting `File > Retrieve from STScI > Retrieve Using Proposal ID` and then entering the program ID (e.g. 1140). (You must be running APT in STScI mode for this retrieval method to be available.)

Export the `.pointing` and `.xml` files for your given proposal in APT by selecting `File > Export...` and selecting both the xml and pointing file options. 

The following APT programs will take place when the mirror segments have not yet been aligned in an image array:
- 1134 (OTE-01: Initial Image Mosaic)
- 1135 (OTE-02: Alignment Matrix 1)
- 1136 (OTE-03: Secondary Mirror Focus Sweep)
- 1137 (OTE-04: Segment ID)
- 1138 (OTE-05: Alignment Matrix 2)

For this example, we are using APT output from a miniaturized version of program 1134 - OTE-01: Initial Image Mosaic. The neccessary files, `OTE01-1134-reduced_mosaic.pointing` and `OTE01-1134-reduced_mosaic.xml`, are located within the `examples/unstacked_mirror_data/` directory.

In [ ]:
# Define the proposal ID
prop_id = 1134

# Where the pointing and XML file for this particular OTE CAR are located
input_dir = './unstacked_mirror_data/'

# Change the root if you named your files differently.
root = 'OTE01-{}-reduced_mosaic'.format(prop_id)
pointing_file = os.path.join(input_dir, '{}.pointing'.format(root))
xml_file = os.path.join(input_dir, '{}.xml'.format(root))

# Make sure those files exist
assert os.path.exists(pointing_file)
assert os.path.exists(xml_file)

### Define location of output files

The process of generating simulated images with MIRaGe produces a lot of files:
- YAML files carrying the OTE mirror state
- YAML files carrying the specifications for simulations
- FITS files of the simulated seed, dark, and compiled images

Additionally, we must create FITS library files of the segment PSF images in order to simulate images with nonnominal PSFs.

Let's define the directories to save these output files to:

In [ ]:
# Where to save MIRaGe output
out_dir = os.path.abspath('./unstacked_mirror_data/output/')

# Where the segment PSF library files will be saved to (and later read from)
library_dir = os.path.join(out_dir, 'gridded_psf_library')

# Make sure both these directories exist
for full_path in [out_dir, library_dir]:
    ensure_dir_exists(full_path)

---
<a id='generate_ote'></a>
# 2. Generate the perturbed OTE state

We outline two methods here to perturb the primary and secondary mirrors, in order to reflect expectations for initial deployment:
- If you already have a YAML file describing a perturbed OTE state that you want to use to generate segment PSFs, use [option 1](#load_ote_from_yaml). (We provide an example file: `unstacked_mirror_data/deployment_errors_reduced_20190521_173912.yaml`)
- Otherwise, if you need to randomly generate a new perturbed OTE state to generate segment PSFs, use [option 2](#random_ote_state).

<a id='load_ote_from_yaml'></a>
### Option 1: Load OTE state from YAML file

To load an OTE state from an existing YAML file, use `mirage.psf.deployments.load_ote_from_deployment_yaml()`.

This function:
- Loads the OTE state definitions from the YAMLfile (piston, tilt, decenter, ROC, and clocking for all 18 primary mirrors and the secondary mirror)
- Applies them to an adjustable `webbpsf.opds.OTE_Linear_Model_WSS` object
- Records the tilt of each segment in a `numpy` array for future reference
- Removes the piston, tip, and tilt so that each individual segment PSF will be be modeled at the center of a detector

In [ ]:
# Load example deployments file
deployments_file = os.path.join(input_dir, 'deployment_errors_reduced_20190521_173912.yaml')
print(deployments_file)
ote, segment_tilts, ote_opd_with_tilts = deployments.load_ote_from_deployment_yaml(
    deployments_file, out_dir
)

In [ ]:
# Display OTE state and tip/tilt vectors
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(ote_opd_with_tilts)
ax1.set_title('OTE with tilts')
ax2.imshow(ote.opd)
ax2.set_title('OTE without tilts')
plt.show()

ote.print_state()

print()
header_string = 'Segment Number |{:^11}|{:^11}'.format('X Tilt', 'Y Tilt')
print(header_string)
print('-' * len(header_string))
for i, (x, y) in enumerate(segment_tilts):
    i_segment = i + 1
    segname = webbpsf.webbpsf_core.segname(i_segment)
    print('{:^14} | {:9.6} | {:9.6}'.format(segname, x, y))

<a id='random_ote_state'></a>
### Option 2: Randomly generate new OTE state

To generate a new, random OTE state, use `mirage.psf.deployments.generate_random_ote_deployment()`.

This function:
- Randomly generates deployment errors for all 18 primary mirrors and the seconday mirror in piston, tilt, decenter, tilt, ROC, and clocking. The magnitude of these errors are pulled from a normal distribution defined by deployment tolerances taken from JWST WFS&C Commissioning and Operations Plan (OTE-24): D36168 / 2299462 Rev C Page 10.
- Applies them to an adjustable `webbpsf.opds.OTE_Linear_Model_WSS` object
- Records the tilt of each segment in a `numpy` array for future reference
- Removes the piston, tip, and tilt so that each individual segment PSF will be be modeled at the center of a detector

We have found, however, that the deployment errors cited in the Ops Plan are rather pessimistic, so for more reasonable OTE states we recommend using the `reduction_factor` argument to reduce the magnitude of the mirror deployments. We use `reduction_factor=0.2` here to reduce the random deployments to 20% of their original values.

In [ ]:
# Randomly generate OTE state with deployment errors
ote, segment_tilts, ote_opd_with_tilts = deployments.generate_random_ote_deployment(
    out_dir, reduction_factor=0.2
)

In [ ]:
# Display OTE state and tip/tilt vectors
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
ax1.imshow(ote_opd_with_tilts)
ax1.set_title('OTE with tilts')
ax2.imshow(ote.opd)
ax2.set_title('OTE without tilts')
plt.show()

ote.print_state()

print()
header_string = 'Segment Number |{:^11}|{:^11}'.format('X Tilt', 'Y Tilt')
print(header_string)
print('-' * len(header_string))
for i, (x, y) in enumerate(segment_tilts):
    i_segment = i + 1
    segname = webbpsf.webbpsf_core.segname(i_segment)
    print('{:^14} | {:9.6} | {:9.6}'.format(segname, x, y))

---
<a id='make_psf_libs'></a>
# 3. Create PSF library files

Once you have a perturbed `webbpsf.opds.OTE_Linear_Model_WSS` object, use the `mirage.psf.segment_psfs.generate_segment_psfs()` function to generate 18 (or more) segment PSF FITS files.

*You can specify multiple detectors and filters with one function call by setting those arguments to be lists of strings (e.g. `['F212N', 'F480M']`). You can also set `detectors='all'` (but not `filters='all'`). Each detector/filter combination which will be saved in a separate file. See [Appendix A](#simulate_whole_obs) for an example of this.*

In [ ]:
# Create PSF libraries
segment_psfs.generate_segment_psfs(ote, segment_tilts, library_dir, 
                                   filters='F212N', detectors='NRCA3', 
                                   overwrite=True)

---
<a id='make_yamls'></a>
# 4. Create YAML files for each exposure

Next, we need to make the YAML files that include all of the parameters for MIRaGe to run.

### Query online catalogs to generate catalog files of sources in FOV

Next, we need to generate catalog files containing RA, Dec, and magnitude for the sources around the target (or targets) in this proposal. 

First we must parse the `.pointing` file to determine the RA and Dec of the target (or targets) that will be observed. Then we will query the appropriate star catalogs to get the magnitudes and locations of shortwave and longwave sources, respectively, around the target(s). If you are using F212N and F480M NIRCam filters, you can use the 2MASS and WISE catalogs, respectively. If you are using one of these two filters, all of this can be accomplished with the `mirage.get_catalog.get_all_catalogs` function.  

If different observations within the proposal have different targets, separate catalogs will be made for each target.

These catalog files will be written to your local `mirage/catalogs/` directory. If files for a given catalog and RA/Dec have already been generated, they will not be regenerated.

<div class="alert alert-block alert-warning">
    
<b>Important:</b> <br>

Querying 2MASS and WISE is only appropriate for observations with the F212N and F480M NIRCam filters. If you want to simulate observations that use other filters, you will have to either query different bandpasses or catalogs or perform a photometric conversion on an existing catalog.

</div>

In [ ]:
# Generate and save shortwave and longwave individual catalogs for the target stars
cats = get_catalog.get_all_catalogs(pointing_file, prop_id)
target_coords, catalog_filenames_sw, catalog_filenames_lw = cats

cat_dict = {'nircam': {'lw': catalog_filenames_lw,
                       'sw': catalog_filenames_sw}}

### Generate the `.yaml` files
Use `mirage.yaml.yaml_generator` to make all of the YAML files for the given APT program - one file per exposure.

**Some additional settings are required to ensure `yaml_generator` works for unstacked mirror simulations.** You must specify the following attributes before running `create_inputs()` to make the YAMLs, and thus simulations, correctly:
- `yam.psf_paths = os.path.expandvars(library_dir)` - tells MIRaGe to look in `library_dir` to find the FITS files for every exposure
- `yam.expand_catalog_for_segments = True` - tells MIRaGe to look for 18 *segment* PSF files rather than one PSF file
- `yam.add_psf_wings = False` - tells MIRaGe not to add wings to the PSF. We don't need wings in this case, since our PSFs are so large.

In [ ]:
# Create a series of data simulator input yaml files from APT files
yaml_dir = os.path.join(out_dir, 'yamls')
yam = yaml_generator.SimInput(input_xml=xml_file, pointing_file=pointing_file,
                              catalogs=cat_dict,
                              verbose=True, output_dir=yaml_dir, simdata_output_dir=out_dir)

# Define the paths to the PSF libraries, and specify to use segment PSFs
yam.psf_paths = os.path.expandvars(library_dir)
yam.expand_catalog_for_segments = True
yam.add_psf_wings = False

# Create all input YAML files (one per each exposure)
yam.create_inputs()

## Choose which exposure to simulate

Now that we've generated all of the needed YAML files, we need to choose one to simulate images with. MIRaGE can only generate one simulated exposure at a time, so we need to choose one YAML file in our `yamls` directory that we will use to produce an image. (See [Appendix A](#simulate_whole_obs) for how use a wrapper to simulate multiple exposures at once with MIRaGe.)

Not every exposure necessarily has the same pointing, so we should choose an exposure that places the target star in the desired detector field-of-view.

### Examine target pointings relative to apertures and V2/V3 references

Looking at the `.pointing` file, let's plot where the target will appear relative to the NIRCam apertures for each unique pointing.

We'll also print out the first YAML in the exposure list for each pointing. In this case, this means printing out six YAML files, for the six different pointings across all the exposures to the "Example files for each pointing" list – each YAML corresponding to an exposure when the pointing changes.

In [ ]:
nc_siaf = pysiaf.Siaf('NIRCam')
nc_full = nc_siaf['NRCA3_FULL']

plt.figure(figsize=(15,10))
for apername in sorted(nc_siaf.apernames):
    a = apername
    if ('_FULL' in a) and ('OSS' not in a) and ('MASK' not in a) and (a[-1] != 'P'):
        nc_siaf[a].plot(frame='tel', label=a, fill_color='white')

# Compare V2/V3 of targets (from .pointing file)
all_pointings = set([(v2, v3, filename) for v2, v3, filename in zip(yam.info['v2'], 
                                                                yam.info['v3'], 
                                                                yam.info['yamlfile']) if 'jw01134001' in filename])

print('Example files for each pointing:')
print('--------------------------------')
plotted_points = []
for i_point, (v2, v3, filename) in enumerate(all_pointings):
    if (v2, v3) not in plotted_points:
        plotted_points.append((v2, v3))
        plt.scatter(v2, v3, marker='*', s=500, 
                    label='Pointing {}/{}'.format(i_point + 1, len(all_pointings)))
        print('{}. {}'.format(i_point + 1, filename))

plt.legend()

plt.show()

### Select the YAML to generate an image from

Looking at the pointing figure above, choose one YAML file that we will create a seed image with MIRaGe for. (Be sure to choose a YAML that has a detector and filter that matches the library files you have created so far.)

*See [JDox](https://jwst-docs.stsci.edu/display/JDAT/File+Naming+Conventions+and+Data+Products) for a detailed explanation of the MIRaGe YAML file name format.*

In [ ]:
# Select one YAML to estimate where the sources will be
test_yaml_filename = 'jw01134001001_01104_00001_nrca3.yaml'
test_yaml = os.path.join(yaml_dir, test_yaml_filename)
print(test_yaml)
yaml_ind = np.where(np.array(yam.info['yamlfile']) == test_yaml_filename)[0][0]

### Calculate and plot V2/V3 locations of NRCA3 aperture, target, and sources

This is the really fun part: by knowing the locations of the source in the selected YAML file and the tilt offsets of each mirror segment PSF, we can plot each segment PSF relative to the selected aperture (here, NRCA3).

In [ ]:
# Examine apertures and V2/V3 references for each array/subarray
nc_siaf = pysiaf.Siaf('NIRCam')
nc_a3 = nc_siaf['NRCA3_FULL']
nc_all = nc_siaf['NRCALL_FULL']

# Get the target RA/Dec
target_ra = float(yam.info['ra'][yaml_ind])
target_dec = float(yam.info['dec'][yaml_ind])

# Get the aperture pointing - THIS IS FOR NRCALL_FULL
pointing_v2 = yam.info['v2'][yaml_ind]
pointing_v3 = yam.info['v3'][yaml_ind]

# Get the aperture RA/Dec - THIS IS FOR NRCA3_FULL
pointing_ra = float(yam.info['ra_ref'][yaml_ind])
pointing_dec = float(yam.info['dec_ref'][yaml_ind])

# Generate the attitude matrix to convert RA/Dec to V2/V3
position_angle = float(yam.info['PAV3'][yaml_ind])
attitude_ref = pysiaf.utils.rotations.attitude(
    pointing_v2, pointing_v3, target_ra, target_dec, position_angle
)

# Get the V2/V3 positions of all sources
sw_catalog_file = yam.info['sw_ptsrc'][yaml_ind]
sw_catalog = asc.read(sw_catalog_file)
v2, v3 = pysiaf.utils.rotations.getv2v3(attitude_ref, sw_catalog['x_or_RA'], sw_catalog['y_or_Dec'])

# Get the V2/V3 position of the target (the same as the NRCALL pointing)
target_v2, target_v3 = pysiaf.utils.rotations.getv2v3(attitude_ref, target_ra, target_dec)

# Get the V2/V3 position of the aperture pointing
a3_v2, a3_v3 = pysiaf.utils.rotations.getv2v3(attitude_ref, pointing_ra, pointing_dec)

# Get library to get offsets for each segment
library_list = segment_psfs.get_segment_library_list('NIRCam', 'NRCA3', 'F212N', library_dir)

In [ ]:
# Plot V2/V3 locations of aperture, target, and sources

fig, ax = plt.subplots(1, 1, figsize=(15,10))
nc_a3.plot(frame='tel', fill_color=None, label='Detector FOV', fill_alpha=0.2)

# Plot the target (the same as the NRCALL pointing)
plt.scatter(target_v2, target_v3, label='Target', marker='*', s=1000, lw=1, 
            edgecolor="black", c='white')

# Plot catalog sources
plt.scatter(v2, v3, label='Catalog Sources', marker='+', s=500, c='black')

# Plot segment sources
for i_segment in np.arange(1, 19):
    marker = 'o'
    if i_segment > 10:
        marker = 'v'
    x_arcsec, y_arcsec = segment_psfs.get_segment_offset(i_segment, 'NRCA3', library_list)
    plt.scatter(v2 - x_arcsec, v3 + y_arcsec, 
                label='Segment {}'.format(i_segment), marker=marker, s=400, alpha=0.6)

# Just show the area close to the detector
plt.xlim(100, -100)
plt.ylim(-600, -400)

# Shrink current axis
box = ax.get_position()
ax.set_position([box.x0, box.y0,
                 box.width * 0.9, box.height])

# Put a legend to the right of the current axis
ax.legend(loc='center left', bbox_to_anchor=(1, 0.5), ncol=1, fontsize=16, facecolor=None)

plt.show()

In this above plot:
- The white star shows the location of the observation's target
- The locations of the 18 different segment PSFs, distributed in a constant pattern around each source, are shown by the colorful circles and triangles
- The black **+** show the locations of stars in the catalog
- The blue square denotes the boundaries of the NRCA3 detector

---
<a id='simulate_images'></a>
# 5. Simulate image with MIRaGe

Finally, we can run MIRaGe to generate a seed image simulation of our unstacked mirror state during OTE-01.

From here on out, from the user perspective, the simulation process is identical to that of nominal imaging cases (see the [imaging example notebook](#Imaging_simulator_use_examples.ipynb). To reiterate, it is the specifications made in the YAML files that enable the simulation of unstacked mirror simulations with MIRaGe.

In [ ]:
# Run the image simulator using the input defined in test_yaml
img_sim = imaging_simulator.ImgSim()
img_sim.paramfile = test_yaml
img_sim.create()

In [ ]:
# Plot the seed image, dark image, and final exposure simulation
fig, [ax1, ax2, ax3] = plt.subplots(1, 3, figsize=(20, 7))
plt.tight_layout()

# Define scale limits and colormap
clim=(1e-2, 1e3)
cmap = cm.get_cmap('viridis')
cmap.set_bad(cmap(0))

# Plot seed image
fitsplot = ax1.imshow(img_sim.seedimage, clim=clim, cmap=cmap)
ax1.set_title('Seed Image', size=24)
ax1.invert_yaxis()

# Plot dark current
ax2.imshow(img_sim.linDark.data[0,-1,:,:] - img_sim.linDark.data[0,0,:,:], clim=clim, cmap=cmap)
ax2.set_title('Dark Current', size=24)
ax2.invert_yaxis()

# Plot final exposure
linear_output = './unstacked_mirror_data/output/jw01134001001_01104_00001_nrca3_linear.fits'
with fits.open(linear_output) as h:
    lindata = h[1].data
    header = h[0].header
exptime = header['EFFINTTM']
diffdata = (lindata[0,-1,:,:] - lindata[0,0,:,:]) / exptime

ax3.imshow(diffdata, clim=clim, cmap=cmap)
ax3.set_title('Final Exposure Simulation', size=24)
ax3.invert_yaxis()

# Define the colorbar
cbar_ax = fig.add_axes([1, 0.09, 0.03, 0.87])
cbar = plt.colorbar(fitsplot, cbar_ax)
cbar.set_label('Count Rate', rotation=270, labelpad=30, size=24)

---
<a id='simulate_whole_obs'></a>
# Appendix A: Simulating many exposures at once

Chances are, you don't want to simulate just one exposure from one detector. In order to simulate all of the exposures from a given observation, write a for loop to iterate over all the YAMLs. We include an example for program 1134 observation 1 below.

### 1. Create all PSF library files
First, make sure that you have created library files for all of the filters and detectors that will be simulated in your observation (for program 1134 observation 1 this means all detectors and filters F212N and F480M):
```python
from mirage.psfs import segment_psfs

segment_psfs.generate_segment_psfs(ote, segment_tilts, library_dir, 
                                   filters=['F212N', 'F480M'], detectors='all')
```

### 2. Run `imaging_simulator` for all YAMLs
Second, grab all of the YAMLs for that observation and run the image simulator on them all.
```python
from mirage import imaging_simulator

# Get all the 1134 Obs 1 NRCA3 yamls
all_yaml_files = glob(os.path.join(yaml_dir, 'jw01134001*.yaml'))
n_yamls = len(all_yaml_files)
print('{} FITS files will be generated.'.format(n_yamls))

for i_yaml, yaml in enumerate(all_yaml_files):
    print('*** SIMULATING YAML {}/{}: {} ***'.format(i_yaml+1, n_yamls, yaml))
    img_sim = imaging_simulator.ImgSim()
    img_sim.paramfile = yaml
    img_sim.create()
```

(If you are impatient and ambitious, you can use Python's `multiprocessing` module to the simulation go faster. Even better on a server with more processors!)

To learn how to combine multiple exposure simulations into a mosaic with QUIP, see [Appendix B](#mosaic).

---
<a id='mosaic'></a>
# Appendix B: Combine into a mosaic

The [`wss_tools`](https://wss-tools.readthedocs.io/en/latest/) include a software program, QUIP, which can be used to combine a list of FITS files into a single mosaic image. QUIP requires an operations file, which we describe how to make here.

### Turn linear FITS products into slope images
```python

from glob import glob
import os

from astropy.io import fits

from mirage.utils.utils import ensure_dir_exists

obs1_fits = glob(os.path.join(out_dir, 'jw*linear.fits'))
print('{} FITS files produced for program APT 1134 Observation 1'.format(len(obs1_fits)))

# Subtract the first from last for each ramp
for f in obs1_fits:
    with fits.open(f) as hdulist:
        data = hdulist[1].data
        hdr = hdulist[1].header
        
    diff = data[0, -1] - data[0, 0]

    hdu = fits.PrimaryHDU(data=diff, header=hdr)

    new_filename = os.path.join(out_dir, 'slope_fits', os.path.basename(f))
    ensure_dir_exists(os.path.dirname(new_filename))
    hdu.writeto(new_filename, overwrite=True)
    
obs1_slope_fits = glob(os.path.join(out_dir, 'slope_fits', 'jw*linear.fits'))
```


### Make ops file for QUIP
```python
# Set variables for writing QUIP ops file
quip_dir = os.path.join(out_dir, 'quip')
ensure_dir_exists(quip_dir)
outfile = 'congrid'
bindim = 2048
opsfile = os.path.join(quip_dir, 'ops_file_'+outfile.strip("/")+str(bindim)+'.xml')

# Write the file
f = open(opsfile, 'w')

f.write('<?xml version="1.0" encoding="UTF-8" standalone="yes"?>\n')
f.write('<QUIP_OPERATION_FILE xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" creator="WSS Executive" time="16:22:40.093Z" date="2017-06-14Z" version="6.0.1" operational="false" xsi:noNamespaceSchemaLocation="/Users/lajoie/TEL/WSS-6.0.1/Software/schema/quip_operation_file.xsd">\n')
f.write('    <CORRECTION_ID>R2017061401</CORRECTION_ID>\n')
f.write('    <OPERATION_TYPE>THUMBNAIL</OPERATION_TYPE>\n')
f.write('    <IMAGES>\n')

for filename in obs1_slope_fits:
    f.write("       <IMAGE_PATH>{:s}</IMAGE_PATH>\n".format(filename))
    
f.write( '       </IMAGES>\n'    )
f.write( '       <OUTPUT>\n')
f.write( '           <OUTPUT_DIRECTORY>{:s}quip/</OUTPUT_DIRECTORY>\n'.format(quip_dir))
f.write( '           <LOG_FILE_PATH>{:s}quip/R2017061401_quip_activity_log.xml</LOG_FILE_PATH>\n'.format(quip_dir))
f.write( '           <OUT_FILE_PATH>{:s}quip/R2017061401_quip_out.xml</OUT_FILE_PATH>\n'.format(quip_dir))
f.write( '       </OUTPUT>\n')

f.write('</QUIP_OPERATION_FILE>\n')

f.close()

print('Successfully wrote QUIP ops file to', opsfile)
```